In [7]:
# get yer imports here

import pandas as pd
import math


In [8]:
# This data is from this website: https://gist.github.com/pramodpendyala/e5688b6a63d2983eac804bbaa1fd7cc0

address_data = pd.read_csv('2023_Gaz_zcta_national.txt', sep="\t", header=None)
address_data.columns = address_data.iloc[0]
address_data = address_data.drop(0)
# Why is the column name like this. Who is responsible. I just want to talk.
address_data.rename(columns={'INTPTLONG                                                                                                                                  ':'INTPTLONG'}, inplace=True)

address_data['INTPTLAT'] = address_data['INTPTLAT'].apply(lambda x: float(x))
address_data['INTPTLONG'] = address_data['INTPTLONG'].apply(lambda x: float(x))

address_data = address_data.drop(["ALAND", "AWATER", "ALAND_SQMI", "AWATER_SQMI"], axis=1)

address_data



,GEOID,INTPTLAT,INTPTLONG
1,00601,18.180555,-66.749961
2,00602,18.361945,-67.175597
3,00603,18.457399,-67.124867
4,00606,18.158327,-66.932928
5,00610,18.295304,-67.125180
...,...,...,...
33787,99923,56.000439,-130.037307
33788,99925,55.550203,-132.945947
33789,99926,55.138352,-131.470425
33790,99927,56.331881,-133.606420


In [9]:
# This data is found from this website: https://warehouse.ninja/target-distribution-center-locations/
# I inputted it manually, unfortunately.

warehouse_data = [[34.6449932755897,-86.8473292167007],
                  [33.44847758394,-112.216516477379],
                  [35.4443517679111,-119.184883986437],
                  [34.1129032695058,-117.481059984653],
                  [34.1556649469121,-117.430952966951],
                  [33.8829312577458,-117.266820757673],
                  [38.6830449523077,-121.721678642205],
                  [34.1342886776624,-117.416355370662],
                  [38.2782732480079,-104.467643668806],
                  [39.8102931481305,-104.986460657673],
                  [30.2323049532679,-82.6359163730191],
                  [31.7765737205048,-81.3526562288365],
                  [31.7776443503854,-81.3539269910744],
                  [31.4215451402305,-83.5182432359064],
                  [32.1739955290448,-81.1755252030619],
                  [21.4792398115305,-158.016842329534],
                  [41.9176226241567,-88.7369320728707],
                  [41.8303900458523,-87.7238539644176],
                  [41.8002418685309,-87.7811810344034],
                  [41.4744170168815,-88.1627609565227],
                  [41.9218023710286,-87.9598046121732],
                  [39.7475498809776,-86.2948367214139],
                  [42.4760732243742,-92.468984016849],
                  [38.9662278505392,-95.6943807344034],
                  [38.8653486027871,-76.7299089768994],
                  [44.9647972364202,-85.2029499259567],
                  [45.0979857630195,-93.2491908864199],
                  [44.9933873277923,-93.2149055171079],
                  [44.9452591409893,-92.9121249623863],
                  [40.5251569148739,-74.2592140725181],
                  [40.089765436428,-74.8200981032102],
                  [39.7817015818632,-75.3450861543144],
                  [43.178721716844,-73.7161892865095],
                  [42.9386799514717,-74.2384685711634],
                  [35.6225966445846,-81.3098264595263],
                  [39.9520393926857,-83.3482983306921],
                  [39.9471432044632,-83.3518352558173],
                  [44.5794790802794,-123.115068328716],
                  [40.1989563009857,-76.8052668801052],
                  [39.8890231958012,-77.6729397],
                  [34.1794119094042,-80.6818378840045],
                  [32.4347483116907,-97.0555079725181],
                  [34.5570298314302,-97.1446958584889],
                  [32.8696311238109,-95.6389149051154],
                  [32.8807284643133,-96.901032357673],
                  [29.9461573473188,-95.5288642134904],
                  [38.0306730381623,-78.998851715346],
                  [36.7076274133422,-76.6651214220913],
                  [47.0821524927726,-122.771626284653],
                  [43.080037381646,-88.4760084558174],
                  [42.4825228934735,-92.4709803136662]]


In [10]:
sample_data = [['17013', 4.69],
               ['22209', 2.49],
               ['19103', 5.29],
               ['07036', 4.19],
               ['21224', 3.89],
               ['12205', 2.69],
               ['10025', 4.59],
               ['16509', 4.89]]

sample_data = pd.DataFrame(sample_data)
sample_data.columns = ["Zip", "Price"]

sample_data = sample_data.merge(right=address_data, left_on="Zip", right_on="GEOID")
sample_data = sample_data.drop("GEOID", axis=1)

def closest_to_distro_center(row):
    distances = []
    
    """
    for coordinate in warehouse_data:
        distance_form_x = pow((row["INTPTLAT"] - coordinate[0]), 2)
        distance_form_y = pow((row["INTPTLONG"] - coordinate[1]), 2)
        distances.append(math.sqrt(distance_form_x + distance_form_y))
    """
    
    for coordinate in warehouse_data:
        # I'll be honest, I have no idea how this works. I don't know most of my trig knowledge anymore, and I am trusting this stackoverflow person with my life.
        # https://stackoverflow.com/questions/365826/calculate-distance-between-2-gps-coordinates
        earth_radius = 3959  # This is earth's radius in miles. I can switch this to fm by changing the number to 6371
        
        distance_lat = math.radians(row["INTPTLAT"] - coordinate[0])
        distance_lon = math.radians(row["INTPTLONG"] - coordinate[1])
        
        lat1 = math.radians(row["INTPTLAT"])
        lat2 = math.radians(coordinate[0])
        
        a = pow(math.sin(distance_lat / 2), 2) + (pow(math.sin(distance_lon / 2), 2) * math.cos(lat1) * math.cos(lat2))
        
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        distances.append(earth_radius * c)
        
    
    return min(distances)


sample_data["Closest Distro Center"] = sample_data.apply(closest_to_distro_center, axis=1)

sample_data



,Zip,Price,INTPTLAT,INTPTLONG,Closest Distro Center
0,17013,4.69,40.241159,-77.183824,20.184734
1,22209,2.49,38.894430,-77.073406,18.585625
2,19103,5.29,39.953372,-75.174187,14.928317
3,07036,4.19,40.624711,-74.239362,6.957404
4,21224,3.89,39.275909,-76.541928,30.107748
5,12205,2.69,42.718696,-73.824087,25.923058
6,10025,4.59,40.798251,-73.968336,24.259755
7,16509,4.89,42.060488,-80.045846,194.499909
